<a href="https://colab.research.google.com/github/moustafa-7/YOLOv3/blob/master/YOLOv3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
os.mkdir("cfg")
os.chdir("cfg")

In [2]:
# download the configuration of the yolov3 network
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg

--2019-06-29 20:59:56--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2019-06-29 20:59:56 (154 MB/s) - ‘yolov3.cfg’ saved [8342/8342]



In [0]:
from __future__ import division

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

In [4]:
!ls

yolov3.cfg


In [0]:
def parse_cfg(cfgfile):
  block = {}
  blocks = []
  
  file = open(cfgfile, 'r')
  lines = file.read().split('\n')
  lines = [x for x in lines if len(x) > 0]
  lines = [x for x in lines if x[0] != '#']              # get rid of comments
  lines = [x.rstrip().lstrip() for x in lines] 
  
  for line in lines:
      if line[0] == "[":               # This marks the start of a new block
          if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
              blocks.append(block)     # add it the blocks list
              block = {}               # re-init the block
          block["type"] = line[1:-1].rstrip()     
      else:
          key,value = line.split("=") 
          block[key.rstrip()] = value.lstrip()
  blocks.append(block)

  return blocks
  

In [0]:
class EmptyLayer(nn.Module):
  def __init__(self):
    super(EmptyLayer, self).__init__()

In [0]:
class DetectionLayer(nn.Module):
  def __init__(self, anchors):
    super(DetectionLayer, self).__init__()
    self.anchors = anchors

In [0]:
def create_modules(blocks):
  net_info  = blocks[0]
  module_list = nn.ModuleList()
  prev_filters = 3
  output_filters = []
  
  for index, x in enumerate(blocks[1:]):
    module = nn.Sequential()
    
   
      
    if x["type"] == "convolutional":
      
       
      activation = x["activation"]
      try:
        batch_normalize = x["batch_normalize"]
        bias = False
      except:
        batch_normalize = 0
        bias = True
      
      
      activation = x["activation"]
      stride = int(x["stride"])
      padding = int(x["pad"])
      kernel_size = int(x["size"])
      filters = int(x["filters"])
          
      if padding:
        pad = (kernel_size - 1) // 2
      else:
        pad = 0
            
      conv = nn.Conv2d(prev_filters, filters, kernel_size = kernel_size, stride = stride, padding = pad, bias = bias)
      module.add_module("conv_{0}".format(index), conv)
    
      if batch_normalize:
        bn = nn.BatchNorm2d(filters)
        module.add_module("bn_{0}".format(index), bn)
      
      
      if activation == "leaky":
        activn = nn.LeakyReLU(0.1, inplace = True)
        module.add_module("leaky_{0}".format(index), activn)
        
        
        
    elif (x["type"] == "upsamling"):
      stride = int(x["stride"])
      upsample = nn.UpSample(scale_factor = 2, mode = "bilinear")
      module.add_module("upsample_{}".format(index), upsample)
      
      
      
    elif (x["type"] == "route"):
      x["layers"] = x["layers"].split(',')
      
      start = int(x["layers"][0])
      
      try:
        end = int(x["layers"][1])
      except:
        end = 0
        
      if start > 0:
        start = start - index
        
      if end >0:
        end = end - index
        
      route = EmptyLayer()
      module.add_module("route_{0}".format(index), route)
      
      if end < 0:
        filters = output_filters[index + start] + output_filters[index + end]
        
      else:
        filters = output_filters[index + start]
        
        
        
        
      
    elif (x["type"] == "shortcut"):
      shortcut = EmptyLayer()
      module.add_module("shortcut_{}".format(index), shortcut)
      
      
      
      
      
    elif (x["type"] == "yolo"):
      mask = x["mask"].split(",")
      mask = [int(x) for x in mask]
      
      anchors = x["anchors"].split(",")
      anchors = [int(a) for a in anchors]
      anchors = [(anchors[i], anchors[i+1]) for i in range(0,len(anchors),2)]
      anchors = [anchors[i] for i in mask]
      
      
      detection = DetectionLayer(anchors)
      module.add_module("Detection_{}".format(index), detection)
      
      
    module_list.append(module)
    prev_filters = filters
    output_filters.append(filters)
    
    
    
  return (net_info, module_list)
      
      
      
      
      
      
      
      
         


In [0]:
blocks = parse_cfg("yolov3.cfg")

In [34]:
print(create_modules(blocks))

({'type': 'net', 'batch': '64', 'subdivisions': '16', 'width': '608', 'height': '608', 'channels': '3', 'momentum': '0.9', 'decay': '0.0005', 'angle': '0', 'saturation': '1.5', 'exposure': '1.5', 'hue': '.1', 'learning_rate': '0.001', 'burn_in': '1000', 'max_batches': '500200', 'policy': 'steps', 'steps': '400000,450000', 'scales': '.1,.1'}, ModuleList(
  (0): Sequential(
    (conv_0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn_0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_0): LeakyReLU(negative_slope=0.1, inplace)
  )
  (1): Sequential(
    (conv_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (leaky_1): LeakyReLU(negative_slope=0.1, inplace)
  )
  (2): Sequential(
    (conv_2): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn_2): BatchNorm2d(32, 